In [33]:
from pinecone import Pinecone as PineconeClient
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda


In [34]:
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
PINECONE_ENVIRONMENT = "aws"
PINECONE_INDEX_NAME ="test001"

In [35]:
pinecone = PineconeClient(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)

In [36]:
embeddings = OpenAIEmbeddings()
vectorstore = Pinecone.from_existing_index(index_name=PINECONE_INDEX_NAME, embedding=embeddings)
retriever = vectorstore.as_retriever()

## RAG Prompt

In [31]:
# RAG prompt
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# RAG
model = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")

chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

In [32]:
chain.invoke("what are minimum viable products?")

'Minimum viable products (MVPs) are versions of a new product that are built with the minimum amount of features necessary to satisfy early adopters and to validate a product concept early in the development cycle. The goal of an MVP is to test, learn, and iterate based on feedback, while minimizing the initial development costs and effort. This approach helps in understanding whether the product being built is valuable to the market and has the potential to be more valuable as it evolves. An MVP typically represents a fraction of the total vision of the final product.'

In [20]:
chain.invoke("what is the product company gap and how can it be solved?")

'The "Product Company Gap" mentioned in the context refers to the challenge of transitioning from a product-focused mindset to building a comprehensive business strategy that encompasses go-to-market plans, pricing, and other elements necessary for scaling a product idea into a successful company. It implies that there is a gap between having a product idea and having a fully-fledged company that can market and sell that product effectively.\n\nWhile the provided context does not give a detailed explanation or solution for the Product Company Gap, it suggests that addressing this gap involves designing products with a broader business perspective from the outset. This includes considering how to market the product, how to price it, and how to build a business model that can sustain and grow the company, rather than focusing solely on the product development aspect.\n\nTo solve the Product Company Gap, one would need to:\n\n1. Integrate go-to-market strategies early in the product devel